1. Реализовать класс DecisionTree. 
В классе **должны** быть методы
```python
def __init__(..., classification:bool=true, ...):
    # some code
def predict(...):
    # some code
def fit(...):
    # some code
```
2. Реализовать класс RandomForest. 
В классе **должны** быть методы
```python
def __init__(..., classification:bool=true, ...):
    # some code
def predict(...):
    # some code
def fit(...):
    # some code
```
3. Реализовать класс GradientBoosting. 
В классе **должны** быть методы
```python
def __init__(..., classification:bool=true, ...):
    # some code
def predict(...):
    # some code
def fit(...):
    # some code
```

4. Обучить модели каждого алгоритма на следующих датасетах (проводим мини-соревнование между ними):
    * регрессия: https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset
    * классификация: https://www.kaggle.com/datasets/pankrzysiu/cifar10-python

5. Объяснить, почему алгоритм $A$ "победил", а почему алгоритм $B$ "проиграл".

## Decision tree

In [1]:
import torch

class DecisionTree:
    def __init__(
        self,
        depth: int = 0,
        max_depth: int = None,
        classification: bool = True,
    ):
        self.depth = depth
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.is_leaf = False
        self.predicted_class = None
        self.classification = classification

    def fit(self, X, y):
        # reset split info each time we fit this node
        self.feature_index = None
        self.threshold = None

        if y.unique().numel() == 1:
            self.is_leaf = True
            self.predicted_class = y[0].item()
            return

        if self.max_depth is not None and self.depth >= self.max_depth:
            self.is_leaf = True
            self.predicted_class = self._get_prediction_class(y)
            return

        if self.classification:
            self._fit_classification(X, y)
        else:
            self._fit_regression(X, y)

    def predict(self, X):
        dtype = torch.long if self.classification else torch.float32

        if self.is_leaf:
            return torch.full((X.shape[0],), self.predicted_class, dtype=dtype)

        left_mask = X[:, self.feature_index] <= self.threshold
        right_mask = X[:, self.feature_index] > self.threshold

        y_pred = torch.empty(X.shape[0], dtype=dtype)
        # cast just in case, to be robust
        y_pred[left_mask] = self.left.predict(X[left_mask]).to(dtype)
        y_pred[right_mask] = self.right.predict(X[right_mask]).to(dtype)

        return y_pred

    def _fit_classification(self, X, y):
        (
            self.feature_index,
            self.threshold,
            best_left_mask,
            best_right_mask,
        ) = self._find_best_split(X, y, self._gini)

        if self.feature_index is None:
            self.is_leaf = True
            self.predicted_class = self._get_prediction_class(y)
            return

        self.left = DecisionTree(
            depth=self.depth + 1,
            max_depth=self.max_depth,
            classification=self.classification,
        )
        self.right = DecisionTree(
            depth=self.depth + 1,
            max_depth=self.max_depth,
            classification=self.classification,
        )

        self.left.fit(X[best_left_mask], y[best_left_mask])
        self.right.fit(X[best_right_mask], y[best_right_mask])

    def _fit_regression(self, X, y):
        (
            self.feature_index,
            self.threshold,
            best_left_mask,
            best_right_mask,
        ) = self._find_best_split(X, y, self._mse)

        if self.feature_index is None:
            self.is_leaf = True
            self.predicted_class = self._get_prediction_class(y)
            return

        self.left = DecisionTree(
            depth=self.depth + 1,
            max_depth=self.max_depth,
            classification=self.classification,
        )
        self.right = DecisionTree(
            depth=self.depth + 1,
            max_depth=self.max_depth,
            classification=self.classification,
        )

        self.left.fit(X[best_left_mask], y[best_left_mask])
        self.right.fit(X[best_right_mask], y[best_right_mask])

    def _find_best_split(self, X, y, criterion_fn):
        """
        Generic split search:
        - criterion_fn(y_left, y_right) -> float (lower is better)
        """
        num_features = X.shape[1]
        best_score = float("inf")
        best_feature = None
        best_threshold = None
        best_left_mask = None
        best_right_mask = None

        for feature in range(num_features):
            thresholds = torch.unique(X[:, feature])

            for ts in thresholds:
                left_mask = X[:, feature] <= ts
                right_mask = X[:, feature] > ts

                # skip degenerate splits
                if not left_mask.any() or not right_mask.any():
                    continue

                y_left = y[left_mask]
                y_right = y[right_mask]

                score = criterion_fn(y_left, y_right)

                if score < best_score:
                    best_score = score
                    best_feature = feature
                    best_threshold = ts.item()
                    best_left_mask = left_mask
                    best_right_mask = right_mask

        return best_feature, best_threshold, best_left_mask, best_right_mask


    def _get_prediction_class(self, y):
        if self.classification:
            return y.mode()[0].item()
        return y.float().mean().item()
        
    @staticmethod
    def _gini_impurity(group):
        if group.numel() == 0:
            return 0.0
        _, counts = torch.unique(group, return_counts=True)
        probs = counts.float() / counts.sum()
        return 1.0 - torch.sum(probs ** 2).item()

    def _gini(self, y_left, y_right):
        total_samples = y_left.numel() + y_right.numel()
        gini_left = self._gini_impurity(y_left)
        gini_right = self._gini_impurity(y_right)

        return (
            (y_left.numel() / total_samples) * gini_left
            + (y_right.numel() / total_samples) * gini_right
        )

    def _mse(self, y_left, y_right):
        total_samples = y_left.numel() + y_right.numel()
        if total_samples == 0:
            return 0.0

        y_left_f = y_left.float()
        y_right_f = y_right.float()

        mse_left = (y_left_f - y_left_f.mean()).pow(2).mean().item()
        mse_right = (y_right_f - y_right_f.mean()).pow(2).mean().item()

        return (
            (y_left.numel() / total_samples) * mse_left
            + (y_right.numel() / total_samples) * mse_right
        )


## Regression

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("london_merged.csv")
data.drop(columns=['timestamp'], inplace=True)

X, y = data.drop(columns=['cnt']).values, data.cnt.values

In [3]:
X_train_np_reg, X_test_np_reg, y_train_np_reg, y_test_np_reg = train_test_split(
    X, y, random_state=42, test_size=0.2
)

In [4]:
X_train_reg = torch.tensor(X_train_np_reg, dtype=torch.float32)
X_test_reg = torch.tensor(X_test_np_reg, dtype=torch.float32)
y_train_reg = torch.tensor(y_train_np_reg, dtype=torch.float32)
y_test_reg = torch.tensor(y_test_np_reg, dtype=torch.float32)


In [5]:
max_depth = 10
dt = DecisionTree(max_depth=max_depth, classification=False)

dt.fit(X_train_reg, y_train_reg)
y_train_pred = dt.predict(X_train_reg)
y_test_pred = dt.predict(X_test_reg)

In [6]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

In [7]:
for m in [r2_score, mean_absolute_error, mean_absolute_percentage_error]:
    print(f'Metric {m.__name__}')
    print(f'Train {m(y_train_np_reg, y_train_pred)}')
    print(f'Test {m(y_test_np_reg, y_test_pred)}')
    print()

Metric r2_score
Train 0.46523517370224
Test 0.20781952142715454

Metric mean_absolute_error
Train 545.825927734375
Test 683.1919555664062

Metric mean_absolute_percentage_error
Train 200039410434048.0
Test 2.636504650115967



## Classification

In [8]:
import pickle
import os

In [9]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='bytes')
    return d

In [10]:
base_dir = 'cifar-10-batches-py'

files = [os.path.join(base_dir, f) for f in os.listdir(base_dir) if '_batch_' in f]

In [11]:
data = [unpickle(f) for f in files]

In [12]:
X_train_np_cl, y_train_np_cl = data[0][b'data'], data[0][b'labels']


In [18]:
y_train_cl.shape

torch.Size([10000])

In [13]:
test_batch = unpickle(os.path.join(base_dir, 'test_batch'))

X_test_np_cl, y_test_np_cl = test_batch[b'data'], test_batch[b'labels']

In [19]:
X_train_cl = torch.tensor(X_train_np_cl, dtype=torch.float32)
X_test_cl = torch.tensor(X_test_np_cl, dtype=torch.float32)
y_train_cl = torch.tensor(y_train_np_cl, dtype=torch.long)
y_test_cl = torch.tensor(y_test_np_cl, dtype=torch.long)

In [19]:
dt_cl = DecisionTree(
    max_depth=1,
    classification=True,
)

print("--------")

dt_cl.fit(
    X_train_cl, y_train_cl
)

--------


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import recall_score, roc_auc_score, precision_score, f1_score, accuracy_score, confusion_matrix

# RandomForest

In [21]:
import numpy as np
import torch

class RandomForest:
    def __init__(
        self,
        n_estimators: int = 100,
        max_depth: int = None,
        max_features = None,
        bootstrap: bool = True,
        classification: bool = True,
        random_state: int | None = None,
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.classification = classification
        self.random_state = random_state
        self.trees = []
        self.features_per_tree = []

    def _get_n_sub_features(self, n_features: int) -> int:
        if self.max_features is None:
            if self.classification:
                k = int(np.sqrt(n_features))
            else:
                k = n_features
        elif isinstance(self.max_features, float):
            k = int(self.max_features * n_features)
        else:
            k = int(self.max_features)
        k = max(1, k)
        k = min(n_features, k)
        return k

    def fit(self, X: torch.Tensor, y: torch.Tensor):
        if self.random_state is not None:
            torch.manual_seed(self.random_state)

        n_samples, n_features = X.shape
        n_sub_features = self._get_n_sub_features(n_features)

        self.trees = []
        self.features_per_tree = []

        for _ in range(self.n_estimators):
            if self.bootstrap:
                indices = torch.randint(0, n_samples, (n_samples,))
            else:
                indices = torch.randperm(n_samples)

            X_sample = X[indices]
            y_sample = y[indices]

            feature_indices = torch.randperm(n_features)[:n_sub_features]
            X_sample_sub = X_sample[:, feature_indices]

            tree = DecisionTree(
                max_depth=self.max_depth,
                classification=self.classification,
            )
            tree.fit(X_sample_sub, y_sample)

            self.trees.append(tree)
            self.features_per_tree.append(feature_indices)

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        if len(self.trees) == 0:
            raise RuntimeError("RandomForest has not been fitted yet.")

        all_preds = []
        for tree, feature_indices in zip(self.trees, self.features_per_tree):
            X_sub = X[:, feature_indices]
            preds = tree.predict(X_sub)
            all_preds.append(preds)

        preds_stack = torch.stack(all_preds, dim=0)

        if self.classification:
            values, _ = torch.mode(preds_stack, dim=0)
            return values

        return preds_stack.float().mean(dim=0)


In [23]:
rf_reg = RandomForest(
    n_estimators=50,
    max_depth=5,
    classification=False,
    random_state=42,
)

rf_reg.fit(X_train_reg, y_train_reg)
y_train_pred = rf_reg.predict(X_train_reg)
y_test_pred = rf_reg.predict(X_test_reg)


In [24]:
for m in [r2_score, mean_absolute_error, mean_absolute_percentage_error]:
    print(f'Metric {m.__name__}')
    print(f'Train {m(y_train_np_reg, y_train_pred)}')
    print(f'Test {m(y_test_np_reg, y_test_pred)}')
    print()

Metric r2_score
Train 0.3291732668876648
Test 0.29936695098876953

Metric mean_absolute_error
Train 648.1138916015625
Test 667.697265625

Metric mean_absolute_percentage_error
Train 196550588366848.0
Test 2.867757797241211



## Gradient boosting

In [27]:
import torch

class GradientBoosting:
    def __init__(
        self,
        n_estimators: int = 100,
        learning_rate: float = 0.1,
        max_depth: int = 3,
        classification: bool = True,
        random_state: int = None,
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.classification = classification
        self.random_state = random_state
        self.trees = []
        self.init_prediction = 0.0

    def fit(self, X: torch.Tensor, y: torch.Tensor):
        if self.random_state is not None:
            torch.manual_seed(self.random_state)

        self.trees = []

        if self.classification:
            y_float = y.float()
            unique = torch.unique(y)
            if unique.numel() != 2:
                raise NotImplementedError("Only binary classification is supported")
            p = y_float.mean()
            eps = 1e-6
            p = torch.clamp(p, eps, 1 - eps)
            self.init_prediction = torch.log(p / (1 - p)).item()
            F = torch.full_like(y_float, self.init_prediction, dtype=torch.float32)
            for _ in range(self.n_estimators):
                p_hat = torch.sigmoid(F)
                residuals = y_float - p_hat
                tree = DecisionTree(
                    max_depth=self.max_depth,
                    classification=False,
                )
                tree.fit(X, residuals)
                update = tree.predict(X).float()
                F = F + self.learning_rate * update
                self.trees.append(tree)
        else:
            y_float = y.float()
            self.init_prediction = y_float.mean().item()
            F = torch.full_like(y_float, self.init_prediction, dtype=torch.float32)
            for _ in range(self.n_estimators):
                residuals = y_float - F
                tree = DecisionTree(
                    max_depth=self.max_depth,
                    classification=False,
                )
                tree.fit(X, residuals)
                update = tree.predict(X).float()
                F = F + self.learning_rate * update
                self.trees.append(tree)

    def _raw_predict(self, X: torch.Tensor) -> torch.Tensor:
        if len(self.trees) == 0:
            raise RuntimeError("GradientBoosting has not been fitted yet.")
        n_samples = X.shape[0]
        F = torch.full((n_samples,), self.init_prediction, dtype=torch.float32)
        for tree in self.trees:
            F = F + self.learning_rate * tree.predict(X).float()
        return F

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        F = self._raw_predict(X)
        if self.classification:
            proba_pos = torch.sigmoid(F)
            return (proba_pos >= 0.5).long()
        return F

    def predict_proba(self, X: torch.Tensor) -> torch.Tensor:
        if not self.classification:
            raise RuntimeError("predict_proba is only available for classification.")
        F = self._raw_predict(X)
        proba_pos = torch.sigmoid(F)
        proba_neg = 1.0 - proba_pos
        return torch.stack([proba_neg, proba_pos], dim=1)
